In [1]:
!pip install transformers[torch] datasets evaluate

In [2]:
!pip install -U datasets

  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/e2/cf/db41e572d7ed958e8679018f8190438ef700aeb501b62da9e1eed9e4d69a/datasets-2.15.0-py3-none-any.whl.metadata
  Obtaining dependency information for pyarrow-hotfix from https://files.pythonhosted.org/packages/e4/f4/9ec2222f5f5f8ea04f66f184caafd991a39c8782e31f5b0266f101cb68ca/pyarrow_hotfix-0.6-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec[http]<=2023.10.0,>=2023.1.0 from https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da228e4763f19e719ac8200752a4841b0b2d/fsspec-2023.10.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.3 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2023.12.2
    Uninstalling fsspec-2023.12.2:
      Successfully uninstalled fsspec-2023.12.2
  Attempting uninstall: 

In [3]:
import pandas as pd
import numpy as np
import json

import os
import random
from tqdm.notebook import tqdm

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader

from transformers import (AutoConfig,
                          AutoTokenizer,
                          DefaultDataCollator,
                          AutoModelForQuestionAnswering,
                          TrainingArguments,
                          Trainer,
                          default_data_collator,
                          get_scheduler)

from datasets import load_dataset
import evaluate
import collections

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = True

seed_everything(42)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

In [8]:
dataset = load_dataset(path='/kaggle/input/aiconnect-mrc',data_files='./custom_data.json',split='train')
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['question', 'answers', 'id', 'context'],
    num_rows: 28024
})

In [9]:
squad = dataset.train_test_split(.2)
squad

DatasetDict({
    train: Dataset({
        features: ['question', 'answers', 'id', 'context'],
        num_rows: 22419
    })
    test: Dataset({
        features: ['question', 'answers', 'id', 'context'],
        num_rows: 5605
    })
})

In [10]:
squad["train"][0]

{'question': '더 나은 삶 지수 중 하나인 공동체 지수에서 어느 나라가 최하위를 기록했니',
 'answers': {'answer_start': [84], 'text': ['대한민국']},
 'id': 'QUES_UniBPiuRnf',
 'context': '경제협력개발기구(OECD)가 발표한 ‘더 나은 삶 지수(Better Life Index, BLI)’ 중 하나인 ‘공동체(Community)’ 지수에서 대한민국은 러시아와 브라질을 포함한 36개 국가 중 최하위를 기록했다. 한국 사회에서 공동체가 파괴되어온 주요인으로 다양한 의견이 개진되었으나, 한국도시의 오래된 특징인 아파트와 고립된 생활 공간, 차도와 인도의 구분이 없는 길과 건물 배치가 이웃 간 소통을 가로막고 갈등이 생겨도 이를 원활하게 해결할 수 없는 환경을 조성했다는 지적이 주목을 받았다(김재형, 2016). 도움이 필요할 때 이웃의 관심과 손길을 기대하기 어려운 환경은 고독사, 아동학대, 가정폭력, 자살 등 다양한 사회문제를 방치하고 악화시키는 데 일조한다. 이러한 공동체 와해와 지역현안 문제를 주민들의 참여를 통해 적극적으로 해결하고자 시행된 정책사업 중 하나가 2010년부터 각 지자체에 본격적으로 도입된 마을 만들기 사업이다. 마을 만들기 사업의 유형은 주체와 내용에 따라 다양한 형태로 발현되지만, 그 핵심은 주민참여를 주요 동력으로 공동의 지역문제와 욕구를 해결하고, 공동체 회복을 통해 주민의 삶의 질을 향상시킨다는 정책적 목적으로 귀결된다(박선희, 2014).'}

In [11]:
print("Context: ", squad["train"][0]["context"])
print("Question: ", squad["train"][0]["question"])
print("Answer: ", squad["train"][0]["answers"])

Context:  경제협력개발기구(OECD)가 발표한 ‘더 나은 삶 지수(Better Life Index, BLI)’ 중 하나인 ‘공동체(Community)’ 지수에서 대한민국은 러시아와 브라질을 포함한 36개 국가 중 최하위를 기록했다. 한국 사회에서 공동체가 파괴되어온 주요인으로 다양한 의견이 개진되었으나, 한국도시의 오래된 특징인 아파트와 고립된 생활 공간, 차도와 인도의 구분이 없는 길과 건물 배치가 이웃 간 소통을 가로막고 갈등이 생겨도 이를 원활하게 해결할 수 없는 환경을 조성했다는 지적이 주목을 받았다(김재형, 2016). 도움이 필요할 때 이웃의 관심과 손길을 기대하기 어려운 환경은 고독사, 아동학대, 가정폭력, 자살 등 다양한 사회문제를 방치하고 악화시키는 데 일조한다. 이러한 공동체 와해와 지역현안 문제를 주민들의 참여를 통해 적극적으로 해결하고자 시행된 정책사업 중 하나가 2010년부터 각 지자체에 본격적으로 도입된 마을 만들기 사업이다. 마을 만들기 사업의 유형은 주체와 내용에 따라 다양한 형태로 발현되지만, 그 핵심은 주민참여를 주요 동력으로 공동의 지역문제와 욕구를 해결하고, 공동체 회복을 통해 주민의 삶의 질을 향상시킨다는 정책적 목적으로 귀결된다(박선희, 2014).
Question:  더 나은 삶 지수 중 하나인 공동체 지수에서 어느 나라가 최하위를 기록했니
Answer:  {'answer_start': [84], 'text': ['대한민국']}


In [12]:
train_dataset = squad["train"]
valid_dataset = squad["test"]

In [13]:
# # select samples
# train_dataset = train_dataset.select(range(1000))
# valid_dataset = valid_dataset.select(range(100))

In [14]:
MODEL_NAME ='monologg/kobigbird-bert-base'
config = AutoConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/492k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

In [15]:
max_length = 1024
stride = 128

def preprocess_training_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride = stride,
        return_offsets_mapping=True,
        return_overflowing_tokens=True,
        padding="max_length",
    )

    #the offset mapping gives us a tuple indicating the sub-token’s start position and end position relative to the original token it was split from
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
      sample_idx = sample_map[i]
      answer = answers[sample_idx]
      start_char = answer["answer_start"][0]
      end_char = answer["answer_start"][0] + len(answer["text"][0])
      sequence_ids = inputs.sequence_ids(i)

      # Find the start and end of the context
      idx = 0
      while sequence_ids[idx] != 1:
          idx += 1
      context_start = idx
      while sequence_ids[idx] == 1:
          idx += 1
      context_end = idx - 1

      # If the answer is not fully inside the context, label it (0, 0)
      if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
          start_positions.append(0)
          end_positions.append(0)
      else:
          # Otherwise it's the start and end token positions
          idx = context_start
          while idx <= context_end and offset[idx][0] <= start_char:
              idx += 1
          start_positions.append(idx - 1)

          idx = context_end
          while idx >= context_start and offset[idx][1] >= end_char:
              idx -= 1
          end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs

In [16]:
def preprocess_validation_function(exmaples):
    questions = [q.strip() for q in exmaples["question"]]
    inputs = tokenizer(
        questions,
        exmaples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(exmaples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [17]:
train_set = train_dataset.map(preprocess_training_function,
                              batched=True,
                              remove_columns= train_dataset.column_names,
                                    load_from_cache_file=False,)

valid_set = valid_dataset.map(preprocess_validation_function,
                              batched=True,
                              remove_columns= valid_dataset.column_names,
                                    load_from_cache_file=False,)

Map:   0%|          | 0/22419 [00:00<?, ? examples/s]

Map:   0%|          | 0/5605 [00:00<?, ? examples/s]

In [18]:
train_set

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 22419
})

In [19]:
valid_set

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 5605
})

In [20]:
valid_modelset =valid_set.remove_columns(["example_id", "offset_mapping"])

In [21]:
train_set.set_format("torch")
valid_modelset.set_format("torch")

In [22]:
Batch_size = 2
trainloader = DataLoader(train_set, shuffle=True, batch_size=Batch_size, collate_fn=default_data_collator)
validloader = DataLoader(valid_modelset, shuffle=False, batch_size=Batch_size, collate_fn=default_data_collator)

In [23]:
metric = evaluate.load("squad")

In [24]:
n_best = 20
max_answer_length = 30

def compute_metrics(start_logits, end_logits,features, examples):
    example_to_features = collections.defaultdict(list)
    for idx, feature in enumerate(features):
        example_to_features[feature["example_id"]].append(idx)

    predicted_answers = []
    for example in tqdm(examples):
        example_id = example["id"]
        context = example["context"]
        answers = []

        for feature_index in example_to_features[example_id]:
            start_logit = start_logits[feature_index]
            end_logit = end_logits[feature_index]
            offsets = features[feature_index]["offset_mapping"]

            start_indexes = np.argsort(start_logit)[-1 : -n_best - 1 : -1].tolist()
            end_indexes = np.argsort(end_logit)[-1 : -n_best - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Skip answers that are not fully in the context
                    if offsets[start_index] is None or offsets[end_index] is None:
                        continue
                    if (end_index < start_index
                        or end_index - start_index +1 > max_answer_length):
                      continue
                    answer = {
                        "text": context[offsets[start_index][0] : offsets[end_index][1]],
                        "logit_score": start_logit[start_index] + end_logit[end_index],}
                    answers.append(answer)

        if len(answers) > 0:
            best_answer = max(answers, key=lambda x: x["logit_score"])
            predicted_answers.append({"id": example_id, "prediction_text": best_answer["text"]})
        else:
            predicted_answers.append({"id": example_id, "prediction_text": ""})
    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    results = metric.compute(predictions=predicted_answers, references=references)
    return results['exact_match'], results['f1']

In [54]:
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

Some weights of BigBirdForQuestionAnswering were not initialized from the model checkpoint at monologg/kobigbird-bert-base and are newly initialized: ['qa_classifier.qa_outputs.weight', 'qa_classifier.output.LayerNorm.bias', 'qa_classifier.output.LayerNorm.weight', 'qa_classifier.intermediate.dense.bias', 'qa_classifier.output.dense.weight', 'qa_classifier.qa_outputs.bias', 'qa_classifier.intermediate.dense.weight', 'qa_classifier.output.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
class EarlyStopping:

  def __init__(self, patience= 3, mode="max", delta=0.0, verbose=True):
    super(EarlyStopping, self).__init__()
    self.early_stop = False
    self.patience = patience
    self.counter = 0
    self.mode = mode
    self.best_score = np.Inf if mode == "min" else 0
    self.delta = delta
    self.verbose = verbose

  def __call__(self, score):
    if self.best_score is None:
      self.best_score = score
      self.counter = 0
    elif self.mode == 'min':
      if score <(self.best_score - self.delta):
        self.counter = 0
        self.best_score = score
        if self.verbose:
           print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
      else:
        self.counter += 1
        if self.verbose:
          print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                f'Best: {self.best_score:.5f}' \
                f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')
    elif self.mode =='max':
        if score > (self.best_score + self.delta):
          self.counter = 0
          self.best_score = score
          if self.verbose:
            print(f'[EarlyStopping] (Update) Best Score: {self.best_score:.5f}')
        else:
          self.counter += 1
          if self.verbose:
            print(f'[EarlyStopping] (Patience) {self.counter}/{self.patience}, ' \
                  f'Best: {self.best_score:.5f}' \
                  f', Current: {score:.5f}, Delta: {np.abs(self.best_score - score):.5f}')

    if self.counter >= self.patience:
      if self.verbose:
        print(f'[EarlyStop Triggered] Best Score: {self.best_score:.5f}')
        self.early_stop = True
      else:
        self.early_stop = False

In [27]:
Learning_rate = 1.858133e-05
num_epochs = 20
num_update_steps_per_epoch = len(trainloader)
num_training_steps = num_epochs * num_update_steps_per_epoch
Weight_decay = 0.176412
accumulation_steps = 16
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

optimizer = AdamW(model.parameters(), lr=Learning_rate)

lr_scheduler = get_scheduler(
    "cosine",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)



In [28]:
def train(trainloader, model, optimizer, lr_scheduler, device):
    model.train()
    running_loss = 0.0
    for idx, batch in enumerate(trainloader):
      batch ={k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      loss = outputs.loss
      running_loss += loss.item()
      if (idx +1) % accumulation_steps == 0:
          loss.backward()
          optimizer.step()
          optimizer.zero_grad()
          lr_scheduler.step()
    train_loss = running_loss / len(trainloader)
    return train_loss

In [29]:
def validate(validloader, model, device):
  model.eval()
  start_logits =[]
  end_logits = []
  with torch.no_grad():
    for idx, batch in enumerate(validloader):
      batch ={k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)

      start_logits.append(outputs.start_logits.detach().cpu().numpy())
      end_logits.append(outputs.end_logits.detach().cpu().numpy())
  start_logits = np.concatenate(start_logits, axis=0)
  end_logits = np.concatenate(end_logits, axis=0)
  results = compute_metrics(start_logits, end_logits, valid_set, valid_dataset)
  return results

In [30]:
best_score = 0.0
early_stopping = EarlyStopping(patience=3, mode='max', verbose=True)

for epoch in tqdm(range(1, num_epochs+1)):
  train_loss = train(trainloader, model, optimizer, lr_scheduler, device)
  em, f1 = validate(validloader, model, device)
  early_stopping(em)
  print("Epoch:{:1d}/{:1d}, Train Loss:{:.4f},  EM:{:.3f}, F1:{:.3f}".format(epoch, num_epochs, train_loss, em, f1 ))
  if em > best_score:
    best_socre = em
    torch.save(model.state_dict(), 'best_model.pt')
  if early_stopping.early_stop:
    print("Early stopping")
    break

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/5605 [00:00<?, ?it/s]

[EarlyStopping] (Update) Best Score: 75.27208
Epoch:1/20, Train Loss:1.5706,  EM:75.272, F1:81.703


  0%|          | 0/5605 [00:00<?, ?it/s]

[EarlyStopping] (Update) Best Score: 79.85727
Epoch:2/20, Train Loss:0.7669,  EM:79.857, F1:85.575


...

[EarlyStopping] (Patience) 1/3, Best: 82.87244, Current: 81.49866, Delta: 1.37377
Epoch:8/20, Train Loss:0.5318,  EM:81.499, F1:87.048


  0%|          | 0/5605 [00:00<?, ?it/s]

[EarlyStopping] (Patience) 2/3, Best: 82.87244, Current: 81.42730, Delta: 1.44514
Epoch:9/20, Train Loss:0.5236,  EM:81.427, F1:87.040


  0%|          | 0/5605 [00:00<?, ?it/s]

[EarlyStopping] (Patience) 3/3, Best: 82.87244, Current: 82.05174, Delta: 0.82070
[EarlyStop Triggered] Best Score: 82.87244
Epoch:10/20, Train Loss:0.5062,  EM:82.052, F1:87.313
Early stopping


In [ ]:
# data = []
# with open('./test.json', encoding="utf-8") as f:
#     squad = json.load(f)
#     for group in squad['data']:
#       for passage in group['paragraphs']:
#           context = passage['context']
#           for qa in passage['qas']:
#               question = qa['question']
#               qu_id = qa['question_id']


#               data.append({
#                 'id':qu_id,
#                 'context':context,
#                 'question':question
#                       })
# with open("custom_test.json", "w") as jsonFile:
#     json.dump(data, jsonFile)

In [42]:
testset = load_dataset(path='/kaggle/input/aiconnect-mrc', data_files='custom_test.json')
testset

DatasetDict({
    train: Dataset({
        features: ['question', 'id', 'context'],
        num_rows: 1626
    })
})

In [55]:
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)
model.load_state_dict(torch.load('/kaggle/working/best_model.pt'))

Some weights of BigBirdForQuestionAnswering were not initialized from the model checkpoint at monologg/kobigbird-bert-base and are newly initialized: ['qa_classifier.qa_outputs.weight', 'qa_classifier.output.LayerNorm.bias', 'qa_classifier.output.LayerNorm.weight', 'qa_classifier.intermediate.dense.bias', 'qa_classifier.output.dense.weight', 'qa_classifier.qa_outputs.bias', 'qa_classifier.intermediate.dense.weight', 'qa_classifier.output.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [59]:
from transformers import pipeline
question_answerer = pipeline("question-answering", model=model, tokenizer=tokenizer)

In [61]:
for data in testset['train']:
    print(data)
    question = data['question']
    context = data['context']
    output = question_answerer(question=question, context=context)
    print(output)
    print(output['answer'])
    break

{'question': '어디서 모나자이트 취급사업장을 생활주변방사선 실태조사 대상으로 뽑았어', 'id': 'QUES_cyOI2451l1', 'context': '생활방사선법 제23조(생활주변방사선 안전관리 실태조사 및 분석) 제1항에 따른 안전관리 실태조사 및 분석은 천연방사성핵종을 함유하는 물질을 취급하는 작업자 보호, 천연방사성핵종을 함유하는 제품을 사용하는 일반인 보호, 국제노선을 운항하는 항공기의 승무원 보호, 수출입 물품 및 재활용고철의 방사선 감시를 통한 방사성물질의 국내 유입 차단 및 오염된 물질의 일반 생활환경으로의 확산 방지를 통한 일반인 보호를 주요 목적으로 하고 있다. 이를 위해 천연방사성핵종 함유물질 및 제품, 재활용고철 감시기 운영 등을 포함한 생활주변방사선 안전관리에 관한 사항에 대하여 매년 안전관리 실태조사·분석 계획을 수립하여 시행하고 그 결과를 통해 안전관리 업무의 효율적 추진을 위한 기초자료 수집이나 제도 개선 사항을 도출하여 차기년도 실태조사에 반영하는 순환적인 업무체계를 갖추고 있다. 한국원자력안전기술원(이하 “KINS”)은 모나자이트 취급사업장과 재활용고철 취급사업장을 2015년도 생활주변방사선 안전관리 실태조사 및 분석(이하 “실태조사”) 대상으로 선정하여 수행하였다.'}
{'score': 0.8501524925231934, 'start': 432, 'end': 442, 'answer': '한국원자력안전기술원'}
한국원자력안전기술원


In [62]:
pred = []
for data in tqdm(testset['train']):
  ques_id = data['id']
  output = question_answerer(question=data['question'], context=data['context'])
  pred.append([ques_id,output['answer']])

  0%|          | 0/1626 [00:00<?, ?it/s]

In [75]:
submission = pd.read_csv('/kaggle/input/aiconnect-mrc/sample_submission.csv')
submission.columns

Index(['question_id', 'answer_text'], dtype='object')

In [84]:
for i in range(len(pred)):
    submission.loc[submission['question_id'] == pred[i][0], 'answer_text'] = pred[i][1]

In [85]:
submission

,question_id,answer_text
0,QUES_cyOI2451l1,한국원자력안전기술원
1,QUES_pz2vbWpWWo,가출청소년
2,QUES_1g3jI4y7eo,탐색기
3,QUES_qzwOZwaeeY,Prime Air라는
4,QUES_hfdtXCtdzf,장애인케어서비스와
...,...,...
1621,QUES_JtsKBSQITG,상용직은
1622,QUES_IajaDLmxvq,공적이전소득
1623,QUES_lR6hjzsptY,대체보육서비스 비용이다
1624,QUES_ACwZJGYBfp,IBM Food Trust Labeyrie사와


In [86]:
submission.to_csv('./submission.csv', index=False)

In [87]:
os.chdir(r'/kaggle/working')

In [88]:
from IPython.display import FileLink
FileLink('./submission.csv')

/kaggle/working/submission.csv